In [1]:
# =============================================================================
# bin/run_model.py
# @author: Brian Kyanjo
# @date: 2024-09-24
# @description: This script runs the flowline model using the EnKF data assimilation scheme.
# =============================================================================

In [2]:
# import libraries ========
import os
import sys
import jax
import warnings
import numpy as np
import jax.numpy as jnp
import matplotlib.pyplot as plt
from scipy.optimize import root
from scipy.stats import norm, multivariate_normal

warnings.filterwarnings("ignore")
jax.config.update("jax_enable_x64", True) # Set the precision in JAX to use float64

In [8]:
# Add the parent and `src` directories to sys.path if needed
for path in [os.path.abspath('..'), os.path.abspath('../src')]:
    if path not in sys.path:
        sys.path.insert(0, path)

# Import necessary modules
from config.config_loader import ParamsLoader
from EnKF.python_enkf.enkf_class_python import EnsembleKalmanFilter
from utils.utils import UtilsFunctions
from flowline_model.flowline_model import FlowlineModel

config_path = os.path.join(os.getcwd(), '../config', 'params.yaml') 

In [12]:
# Initial setup from the config file and initial guess ---------------
params = ParamsLoader(config_path).get_params()
grid = params['grid']

# Create instances of the necessary classes
utils_functions = UtilsFunctions(params)
flowline_model  =  FlowlineModel(params,grid,utils_functions.bed)

# Initial guess and steady state ====================
xg = 300e3 / params["xscale"]
hf = (-utils_functions.bed(xg * params["xscale"]) / params["hscale"]) / (1 - params["lambda"])
h  = 1 - (1 - hf) * grid["sigma"]
u  = 1.0 * (grid["sigma_elem"] ** (1 / 3)) + 1e-3
huxg_old = np.concatenate((h, u, [xg]))

# Run the flowline model ====================
Jf = flowline_model.jacobian(huxg_old,flowline_model.flowline) # Jacobian of the model

# solve the system of equations
solve_result = root(
    lambda varin: flowline_model.flowline(varin,huxg_old), 
    huxg_old, 
    jac=Jf,
    method="hybr",
    options={"xtol": 1e-12, "maxfev": 10000},
)
huxg_out0 = solve_result.x # extract the solution
huxg_out0

KeyError: 'NX'

In [13]:
params["NX"]

KeyError: 'NX'